In [37]:
import plotly.express as px
import pandas as pd
import json
import datetime
from pytimeparse.timeparse import timeparse
from datetime import timedelta

# Load rostering data
with open(f'./rostering.json', 'r', encoding='utf-8') as f:
        rostering = json.load(f)

# Load rostering data
with open(f'./meta_file.json', 'r', encoding='utf-8') as f:
        meta = json.load(f)

shifts_duration = {}
for s in meta['shifts']:
    shifts_duration[s['id']] = datetime.timedelta(minutes= int(s['duration'][:-3]) * 60+ int(s['duration'][-2:]))

print(shifts_duration)

{'500c1e07-031d-42e4-952d-658b8617523a': datetime.timedelta(seconds=32400), '37ebd385-adfe-4309-aa49-4ca559221624': datetime.timedelta(seconds=32400), '62292b5f-2887-4244-a751-41139579e801': datetime.timedelta(seconds=43200), 'a6dc6208-87ef-4eb0-8d4e-04671ef101fa': datetime.timedelta(seconds=32400), '272aa166-b628-4e80-a75b-d4f2ddb96655': datetime.timedelta(seconds=43200), '72fb0fbb-b48e-40ed-8489-dcc8a3a7c71a': datetime.timedelta(seconds=43200), '8f8b05c0-6670-499e-82dd-e146ed23a967': datetime.timedelta(seconds=43200), 'd1a807a2-a6e3-42a9-8236-c870b4a21a84': datetime.timedelta(seconds=43200), 'd59217b1-dea5-4af4-983e-725fda26f65a': datetime.timedelta(seconds=43200), '977869f6-8641-4256-93a8-f9fe7a7de958': datetime.timedelta(seconds=43200), '8b73ad1c-f897-4c0c-bb45-936ea7c8e71b': datetime.timedelta(seconds=32400), '31a095d6-6c31-4656-bb43-997834460c69': datetime.timedelta(seconds=32400), '8a33d704-0131-445a-940b-ff7a31f7b966': datetime.timedelta(seconds=43200), '6fbb4021-245e-4b61-a94b

In [2]:
# shifts to dataframe

format = '%d.%m.%y %H:%M'
shifts = map(
        lambda x: dict(Task = str(x['employeeId']),
                       Start = datetime.datetime.strptime(f"{x['shiftDate']} {x['shiftTimeStart']}", format),
                       Finish = datetime.datetime.strptime(f"{x['shiftDate']} {x['shiftTimeStart']}", format) + shifts_duration[x['shiftId']],
                       Resource = x['shiftId'])
        , rostering['campainSchedule']
)

# breaks

df_shifts = pd.DataFrame(shifts)
df_shifts.head()

,Task,Start,Finish,Resource
0,0006aabe-0ba4-7a35-c0b0-bf6596f85159,2023-03-01 06:00:00,2023-03-01 15:00:00,8b73ad1c-f897-4c0c-bb45-936ea7c8e71b
1,0006aabe-0ba4-7a35-c0b0-bf6596f85159,2023-03-02 06:45:00,2023-03-02 15:45:00,8b73ad1c-f897-4c0c-bb45-936ea7c8e71b
2,0006aabe-0ba4-7a35-c0b0-bf6596f85159,2023-03-03 06:00:00,2023-03-03 15:00:00,8b73ad1c-f897-4c0c-bb45-936ea7c8e71b
3,0006aabe-0ba4-7a35-c0b0-bf6596f85159,2023-03-04 06:00:00,2023-03-04 15:00:00,8b73ad1c-f897-4c0c-bb45-936ea7c8e71b
4,0006aabe-0ba4-7a35-c0b0-bf6596f85159,2023-03-05 06:00:00,2023-03-05 15:00:00,8b73ad1c-f897-4c0c-bb45-936ea7c8e71b


In [ ]:
# breaks to dataframe
breaks = []

for s in rostering['campainSchedule']:
    for a in s['activities']:
        employee_id = s['employeeId']
        activity_id = a['activityId']

        shift_start = datetime.datetime.strptime(f"{s['shiftDate']} {s['shiftTimeStart']}", format)
        activity_start = datetime.datetime.strptime(f"{s['shiftDate']} {a['activityTimeStart']}", format)
        activity_end = datetime.datetime.strptime(f"{s['shiftDate']} {a['activityTimeEnd']}", format)

        if activity_end < activity_start:
            # if activity start time < shift start time => its overnight activity and just +1 day
            activity_end += datetime.timedelta(days=1)

        if activity_start < shift_start:
            # if activity start time < shift start time => its overnight activity and just +1 day
            activity_start += datetime.timedelta(days=1)
            activity_end += datetime.timedelta(days=1)

        breaks.append(
            dict(Task = str(employee_id),
                 Start = activity_start,
                 Finish = activity_end,
                 Resource = activity_id)
        )

df_breaks = pd.DataFrame(breaks)
df_breaks.head()

In [ ]:
df = pd.concat([df_shifts, df_breaks])
color_map = {
    '12 часов день обед 1': '#000000',
    '12 часов день обед 2': '#000000',
    '12 часов ночь обед': '#000000',
    '9 часов день обед': '#000000',
    '9 часов ночь обед': '#000000',
    '12 часов ночь перерыв 1': '#FFFFFF',
    '12 часов ночь перерыв 2': '#FFFFFF',
    '12 часов ночь перерыв 3': '#FFFFFF',
    '12 часов ночь перерыв 4': '#FFFFFF',
    '12 часов день перерыв 1': '#FFFFFF',
    '12 часов день перерыв 2': '#FFFFFF',
    '9 часов день перерыв 1': '#FFFFFF',
    '9 часов день перерыв 2': '#FFFFFF',
    '9 часов ночь перерыв 1': '#FFFFFF',
    '9 часов ночь перерыв 2': '#FFFFFF',
    '9 часов ночь перерыв 3 личн': '#FFFFFF'
}

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", color="Resource", color_discrete_map = color_map, height = 10000)
fig.update_yaxes(autorange="reversed")
# fig.update_xaxes(rangeslider_visible=True)
fig.show()


In [ ]:
# pip install --upgrade "kaleido==0.1.*"
import kaleido
import plotly.graph_objects as go
print(kaleido.__version__)

fig_export = px.timeline(df, x_start="Start", x_end="Finish", y="Task", color="Resource", color_discrete_map = color_map, height = 10000, width = 10000)
fig_export.update_yaxes(autorange="reversed")

fig_export.write_image("C:\\Projects\inline-pro\\pyworkforce\\examples\\rostering_schedule.png")

## Compare SL statistics

The snippet below loads statistics results and create a plot for further comparison. Could be useful to compare different algorithm versions

In [3]:
# show slas

# Load stats data
with open(f'./statistics_output_old.json', 'r', encoding='utf-8') as f:
        statisticss = json.load(f)


shifts_duration = {}
for s in meta['shifts']:
    shifts_duration[s['id']] = datetime.timedelta(minutes= int(s['duration'][:-3]) * 60+ int(s['duration'][-2:]))


# shifts to dataframe

format = '%d.%m.%y %H:%M'
shifts = map(
        lambda x: dict(Task = str(x['employeeId']),
                       Start = datetime.datetime.strptime(f"{x['shiftDate']} {x['shiftTimeStart']}", format),
                       Finish = datetime.datetime.strptime(f"{x['shiftDate']} {x['shiftTimeStart']}", format) + shifts_duration[x['shiftId']],
                       Resource = x['shiftId'])
        , rostering['campainSchedule']
)

# breaks

df_shifts = pd.DataFrame(shifts)
df_shifts.head()

,Task,Start,Finish,Resource
0,0006aabe-0ba4-7a35-c0b0-bf6596f85159,2023-03-01 06:00:00,2023-03-01 15:00:00,8b73ad1c-f897-4c0c-bb45-936ea7c8e71b
1,0006aabe-0ba4-7a35-c0b0-bf6596f85159,2023-03-02 06:45:00,2023-03-02 15:45:00,8b73ad1c-f897-4c0c-bb45-936ea7c8e71b
2,0006aabe-0ba4-7a35-c0b0-bf6596f85159,2023-03-03 06:00:00,2023-03-03 15:00:00,8b73ad1c-f897-4c0c-bb45-936ea7c8e71b
3,0006aabe-0ba4-7a35-c0b0-bf6596f85159,2023-03-04 06:00:00,2023-03-04 15:00:00,8b73ad1c-f897-4c0c-bb45-936ea7c8e71b
4,0006aabe-0ba4-7a35-c0b0-bf6596f85159,2023-03-05 06:00:00,2023-03-05 15:00:00,8b73ad1c-f897-4c0c-bb45-936ea7c8e71b


In [49]:
df_stats = pd.read_json(f'./statistics_output.json')

df_stats['tc'] = pd.to_datetime(df_stats['tc'])
df_stats.set_index('tc', inplace=False)

,call_volume,aht,service_level,art,positions,scheduled_positions,scheduled_service_level,scheduled_asa,zero_level_positions
tc,,,,,,,,,
2023-03-01 00:00:00+03:00,15,413.866667,80,30,10,5,0.00,-461.16,8
2023-03-01 00:15:00+03:00,11,421.000000,80,30,8,5,0.00,-3106.90,6
2023-03-01 00:30:00+03:00,13,410.000000,80,30,9,5,0.00,-668.75,7
2023-03-01 00:45:00+03:00,9,435.000000,80,30,7,5,65.29,466.31,5
2023-03-01 01:00:00+03:00,13,434.000000,80,30,9,5,0.00,-585.90,7
...,...,...,...,...,...,...,...,...,...
2023-03-31 22:45:00+03:00,6,355.000000,80,30,5,4,79.03,60.10,3
2023-03-31 23:00:00+03:00,7,375.000000,80,30,5,5,82.60,38.88,4
2023-03-31 23:15:00+03:00,4,324.000000,80,30,3,5,98.31,1.56,2


In [11]:
fig = px.line(x="tc", y="scheduled_service_level", data_frame=df_stats, title="scheduled_service_level")
fig.update_layout(autosize=True)
# fig.update_layout(width=1000)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [44]:
fig = px.line(x="tc", y=["positions", "scheduled_positions"], data_frame=df_stats, title="positions vs scheduled positions")
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [51]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)
                    # specs=[[{"secondary_y": True}, {"secondary_y": True}],
                    #        [{"secondary_y": True}, {"secondary_y": True}]])

# 1st graph - just service level
fig.add_trace(
    go.Scatter(x=df_stats['tc'], y=df_stats['scheduled_service_level'], name="Service level"),
    row=1, col=1)

# 2nd graph - required positions + scheduled positions
fig.add_trace(
    go.Scatter(x=df_stats['tc'], y=df_stats['positions'], name="Required positions"),
    row=2, col=1) # , secondary_y=False)

fig.add_trace(
    go.Scatter(x=df_stats['tc'], y=df_stats['scheduled_positions'], name="Scheduled positions"),
    row=2, col=1) # , secondary_y=False)

# fig.update_layout(autosize=True)

# sync scrolling
fig.update_xaxes(matches='x')

# only 1 slider at the bottom
fig.update_xaxes(rangeslider_visible=True)
fig.update_xaxes(row=1, col=1, rangeslider_visible=False)

fig.update_layout(height=800)
fig.show()